In [1]:
import csv
import json
import os
import glob
import shutil
import pickle as pkl
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm, trange
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pathlib import Path
import dask.dataframe as dd

In [6]:
# GPU 사용 가능 여부 확인
if tf.config.list_physical_devices('GPU'):
    print("GPU가 사용 가능합니다.")
    print("사용 가능한 GPU 목록:", tf.config.list_physical_devices('GPU'))
else:
    print("GPU를 사용할 수 없습니다.")

GPU가 사용 가능합니다.
사용 가능한 GPU 목록: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# **JSON Merged**

In [ ]:
# 데이터셋 경로
DATA_PATH = r"F:\Datasets"

In [10]:
# 모든 JSON 파일 경로 수집 (Other 폴더 제외)
json_files = []
for root, dirs, files in os.walk(DATA_PATH):
    # "Other" 폴더를 제외
    dirs[:] = [d for d in dirs if d != "Other"]
    for file in files:
        if file.endswith(".json"):
            json_files.append(os.path.join(root, file))

print(f"총 {len(json_files)}개의 JSON 파일을 발견했습니다.")

총 4613074개의 JSON 파일을 발견했습니다.


In [24]:
def merge_csv_and_json(data_path, output_path, chunk_size=30000):
    """
    Training 및 Validation 폴더를 순회하며 JSON과 CSV 파일을 병합하여 새로운 CSV 파일 생성
    데이터를 chunk_size 단위로 나눠서 저장

    Parameters:
    - data_path (str): Datasets 루트 경로 (Training 및 Validation 폴더 포함)
    - output_path (str): 병합 결과를 저장할 디렉토리 경로
    - chunk_size (int): 한 번에 저장할 데이터 행 수

    Returns:
    - None
    """
    # JSON 및 CSV 파일 수집
    json_files = {}
    csv_files = {}

    for root, dirs, files in os.walk(data_path):
        # "Other" 폴더 제외
        dirs[:] = [d for d in dirs if d != "Other"]
        for file in files:
            if file.endswith(".json"):
                file_key = os.path.basename(file).replace('.json', '')
                json_files[file_key] = os.path.join(root, file)
            elif file.endswith(".csv"):
                file_key = os.path.basename(file).replace('.csv', '')
                csv_files[file_key] = os.path.join(root, file)

    # 공통 파일 이름 확인
    common_files = set(json_files.keys()) & set(csv_files.keys())
    print(f"Training 및 Validation 폴더에서 매칭된 파일 개수: {len(common_files)}")

    # 병합 결과 저장
    combined_data = []
    
    # 병합된 파일 개수 기준으로 저장
    file_count = 0
    chunk_index = 1

    for file_name in tqdm(common_files, desc="Merging CSV and JSON files"):
        try:
            # JSON 파일 읽기
            with open(json_files[file_name], 'r', encoding='utf-8') as f:
                json_data = json.load(f)

            # JSON 데이터를 평탄화
            json_df = pd.json_normalize(json_data)

            # CSV 파일 읽기
            csv_df = pd.read_csv(csv_files[file_name])

            # JSON 데이터를 CSV 행 개수에 맞게 복사
            json_df_repeated = pd.concat([json_df] * len(csv_df), ignore_index=True)

            # JSON 데이터와 CSV 데이터 병합
            merged_df = pd.concat([json_df_repeated, csv_df.reset_index(drop=True)], axis=1)
            combined_data.append(merged_df)
            file_count += 1

            # 파일 개수 기준으로 저장
            if file_count >= chunk_size:
                final_df = pd.concat(combined_data, ignore_index=True)
                output_file = os.path.join(output_path, f"merged_data_{chunk_index:02d}.csv")
                final_df.to_csv(output_file, index=False)
                print(f"병합된 데이터를 저장했습니다: {output_file}")
                combined_data = []
                file_count = 0
                chunk_index += 1

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

    # 남은 데이터 저장
    if combined_data:
        final_df = pd.concat(combined_data, ignore_index=True)
        output_file = os.path.join(output_path, f"merged_data_{chunk_index:02d}.csv")
        final_df.to_csv(output_file, index=False)
        print(f"병합된 데이터를 저장했습니다: {output_file}")

    print("모든 병합 작업이 완료되었습니다.")


In [25]:
merge_csv_and_json(DATA_PATH, r'D:\repos\V2X_Incident_Prediction\merge')

Training 및 Validation 폴더에서 매칭된 파일 개수: 4613074


Merging CSV and JSON files:   1%|          | 29997/4613074 [04:18<10:27:13, 121.78it/s]

병합된 데이터를 저장했습니다: D:\repos\V2X_Incident_Prediction\merge\merged_data_01.csv


Merging CSV and JSON files:   1%|▏         | 59992/4613074 [08:55<9:58:51, 126.72it/s] 

병합된 데이터를 저장했습니다: D:\repos\V2X_Incident_Prediction\merge\merged_data_02.csv


...

병합된 데이터를 저장했습니다: D:\repos\V2X_Incident_Prediction\merge\merged_data_152.csv


Merging CSV and JSON files:  99%|█████████▉| 4589995/4613074 [11:19:29<03:39, 105.01it/s] 

병합된 데이터를 저장했습니다: D:\repos\V2X_Incident_Prediction\merge\merged_data_153.csv


Merging CSV and JSON files: 100%|██████████| 4613074/4613074 [11:23:01<00:00, 112.56it/s]


병합된 데이터를 저장했습니다: D:\repos\V2X_Incident_Prediction\merge\merged_data_154.csv
모든 병합 작업이 완료되었습니다.


In [26]:
def merge_csv_files(input_path, output_file):
    """
    주어진 디렉토리 내의 모든 CSV 파일을 하나로 병합하여 저장

    Parameters:
    - input_path (str): CSV 파일들이 저장된 디렉토리 경로
    - output_file (str): 병합된 CSV 파일을 저장할 경로

    Returns:
    - None
    """
    csv_files = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.csv')]
    print(f"발견된 CSV 파일 개수: {len(csv_files)}")

    combined_data = []
    for file in tqdm(csv_files, desc="Merging CSV files"):
        try:
            df = pd.read_csv(file)
            combined_data.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    if combined_data:
        merged_df = pd.concat(combined_data, ignore_index=True)
        merged_df.to_csv(output_file, index=False)
        print(f"병합된 데이터를 저장했습니다: {output_file}")
    else:
        print("병합할 CSV 파일이 없습니다.")

In [27]:
# 경로 설정
input_path = "merge"       # 병합할 CSV 파일들이 있는 폴더 경로
output_file = "merge.csv"  # 병합된 파일을 저장할 경로

# 함수 실행
merge_csv_files(input_path, output_file)

발견된 CSV 파일 개수: 154


Merging CSV files: 100%|██████████| 154/154 [00:43<00:00,  3.50it/s]


병합된 데이터를 저장했습니다: merge.csv


In [29]:
# 저장된 파일 불러오기
df = pd.read_csv('merge.csv')
df.head(10)

,Path,Annotation.Vehicle_ID,Annotation.Turn,Annotation.Lane,Annotation.Speed,Annotation.Hazard,ISSUE_DATE,VEHICLE_ID,VEHICLE_CLASS,LONGITUDE,LATITUDE,HEADING,SPEED,BRAKE_STATUS,ACC_SEC,VEHICLE_TYPE,CURRENT_LANE,Annotation.Index
0,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153241,FFE80BAA,9228.0,126.916046,35.162495,287.0,38.0,78.0,0.0,bus,0,NaN
1,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153242,FFE80BAA,9228.0,126.915924,35.162526,286.0,37.0,78.0,-28.0,bus,0,NaN
2,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153243,FFE80BAA,9228.0,126.915827,35.162551,287.0,34.0,78.0,-56.0,bus,0,NaN
3,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153244,FFE80BAA,9228.0,126.915727,35.162578,288.0,31.0,78.0,-111.0,bus,0,NaN
4,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153245,FFE80BAA,9228.0,126.915636,35.162603,288.0,29.0,78.0,-83.0,bus,0,NaN
5,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153246,FFE80BAA,9228.0,126.915555,35.162625,288.0,24.0,78.0,-111.0,bus,0,NaN
6,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153247,FFE80BAA,9228.0,126.915494,35.162643,290.0,16.0,78.0,-194.0,bus,0,NaN
7,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153248,FFE80BAA,9228.0,126.915459,35.162654,289.0,7.0,78.0,-222.0,bus,0,NaN
8,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153249,FFE80BAA,9228.0,126.915448,35.162657,289.0,0.0,78.0,-194.0,bus,4,NaN
9,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153250,FFE80BAA,9228.0,126.915448,35.162657,289.0,0.0,78.0,0.0,bus,4,NaN


In [32]:
# 칼럼명 변경
df.columns = [col.replace("Annotation.", "") for col in df.columns]
df.drop(columns=['Index'])

,Path,Vehicle_ID,Turn,Lane,Speed,Hazard,ISSUE_DATE,VEHICLE_ID,VEHICLE_CLASS,LONGITUDE,LATITUDE,HEADING,SPEED,BRAKE_STATUS,ACC_SEC,VEHICLE_TYPE,CURRENT_LANE
0,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153241,FFE80BAA,9228.0,126.916046,35.162495,287.0,38.0,78.0,0.0,bus,0
1,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153242,FFE80BAA,9228.0,126.915924,35.162526,286.0,37.0,78.0,-28.0,bus,0
2,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153243,FFE80BAA,9228.0,126.915827,35.162551,287.0,34.0,78.0,-56.0,bus,0
3,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153244,FFE80BAA,9228.0,126.915727,35.162578,288.0,31.0,78.0,-111.0,bus,0
4,V_220911_C_A_FFE80BAA_000433.csv,FFE80BAA,False,False,False,False,20220911153245,FFE80BAA,9228.0,126.915636,35.162603,288.0,29.0,78.0,-83.0,bus,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46130725,V_221008_C_A_FFE80B4C_000374.csv,FFE80B4C,False,False,False,False,20221008160145,FFE80B4C,9228.0,126.880411,35.161630,266.0,21.0,0.0,56.0,bus,0
46130726,V_221008_C_A_FFE80B4C_000374.csv,FFE80B4C,False,False,False,False,20221008160146,FFE80B4C,9228.0,126.880341,35.161628,270.0,23.0,0.0,83.0,bus,0
46130727,V_221008_C_A_FFE80B4C_000374.csv,FFE80B4C,False,False,False,False,20221008160147,FFE80B4C,9228.0,126.880263,35.161629,271.0,27.0,0.0,83.0,bus,0
46130728,V_221008_C_A_FFE80B4C_000374.csv,FFE80B4C,False,False,False,False,20221008160148,FFE80B4C,9228.0,126.880177,35.161633,272.0,29.0,0.0,56.0,bus,0


In [34]:
# ISSUE_DATE 칼럼을 DateTime 형식으로 변환
df['ISSUE_DATE'] = pd.to_datetime(df['ISSUE_DATE'], format='%Y%m%d%H%M%S')
df['ISSUE_DATE']

0          2022-09-11 15:32:41
1          2022-09-11 15:32:42
2          2022-09-11 15:32:43
3          2022-09-11 15:32:44
4          2022-09-11 15:32:45
                   ...        
46130725   2022-10-08 16:01:45
46130726   2022-10-08 16:01:46
46130727   2022-10-08 16:01:47
46130728   2022-10-08 16:01:48
46130729   2022-10-08 16:01:49
Name: ISSUE_DATE, Length: 46130730, dtype: datetime64[ns]

In [36]:
df.to_csv('merge.csv', index=False)